In [1]:
import findspark
findspark.init('/home/mysparkub/spark-3.0.0-bin-hadoop2.7')
findspark.init('/home/mysparkub/spark-3.0.0-bin-hadoop2.7')

In [24]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import LinearRegression

In [3]:
spark = SparkSession.builder.appName('cruise_ship').getOrCreate()

In [4]:
data = spark.read.csv('files/cruise_ship_info.csv', inferSchema=True, header=True)

In [8]:
for i in data.head(1)[0]:
    print(i)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


In [11]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [10]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [29]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='cruise_line_int')
data = indexer.fit(data).transform(data)

In [28]:
indexed.head(2)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_line_int=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_line_int=16.0)]

In [30]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- cruise_line_int: double (nullable = false)



In [31]:
assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers',
                                       'length','cabins', 'passenger_density', 'cruise_line_int'],
                            outputCol='features'
                           )

In [32]:
output = assembler.transform(data)

In [33]:
final_data = output.select('features', 'crew')

In [52]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [53]:
lr = LinearRegression(labelCol='crew')

In [54]:
lr_model = lr.fit(train_data)

In [55]:
test_result = lr_model.evaluate(test_data)

In [56]:
# previous : 0.9029397156598549
test_result.r2

0.9684141418700678

In [57]:
# previous : 1.029040635667725
test_result.rootMeanSquaredError

0.6733813368424998